> This notebook presents the preambles needed for working with Gym at Google CoLab and Gym Environments. 

# CoLab Preambles

Most of the requirements of python packages are already fulfilled on CoLab. To run Gym, you have to install prerequisites like xvbf,opengl & other python-dev packages using the following codes.

In [ ]:
#!apt-get update --fix-missing
!pip install -q gym
!apt-get install python-opengl -y
!apt-get install -y xvfb x11-utils

Get atari game roms

In [ ]:
!wget http://www.atarimania.com/roms/Roms.rar

In [ ]:
!unrar x /content/Roms.rar

In [ ]:
!unzip /content/ROMS.zip

In [ ]:
!pip install gym-retro
!python3 -m retro.import ROMS/

For rendering environment, you can use pyvirtualdisplay. So fulfill that 

In [ ]:
!pip install pyvirtualdisplay
!pip install piglet

To activate virtual display we need to run a script once for training an agent, as follows:

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [8]:
# This code creates a virtual display to draw game images on. 
# If you are running locally, just ignore it
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1

In [9]:
import retro
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) # error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [10]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

# OpenAI Gym Available Environment

Gym comes with a diverse suite of environments that range from easy to difficult and involve many different kinds of data. View the [full list of environments](https://gym.openai.com/envs) to get the birds-eye view.

- [Classic control](https://gym.openai.com/envs#classic_control) and [toy text](https://gym.openai.com/envs#toy_text): complete small-scale tasks, mostly from the RL literature. They’re here to get you started.

- [Algorithmic](https://gym.openai.com/envs#algorithmic): perform computations such as adding multi-digit numbers and reversing sequences. One might object that these tasks are easy for a computer. The challenge is to learn these algorithms purely from examples. These tasks have the nice property that it’s easy to vary the difficulty by varying the sequence length.

- [Atari](https://gym.openai.com/envs#atari): play classic Atari games. 

- [2D and 3D robots](https://gym.openai.com/envs#mujoco): control a robot in simulation. These tasks use the MuJoCo physics engine, which was designed for fast and accurate robot simulation. 


##  List the Environments Available in your Installation

gym’s main purpose is to provide a large collection of environments that expose a common interface and are versioned to allow for comparisons. 

In [ ]:
from gym import envs
print(envs.registry.all())

## Algorithmic

These are a variety of algorithmic tasks, such as learning to copy a sequence.

In [ ]:
env = gym.make('Copy-v0')
env.reset()
#plt.imshow(env.render())
env.render()

## Atari

The Atari environments are a variety of Atari video games. Gym is already installed but not with atari game environments, to get that:

In [ ]:
!pip install gym[atari]

In [ ]:
# Atari Environment
env = retro.make('SpaceInvaders-Atari2600')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

episodes =5

for episode in range(1, episodes):
  state = env.reset()
  done = False
  score = 0

  while not done:
    env.render(mode='rgb_array')
    action = random.choice([0,1,2,3,4,5,6])
    n_state, reward, done, info = env.step([action])
    score += reward
  print(f"Episode : {episode} score : {score}")
  plt.imshow(env.render('rgb_array'))
env.close()


## Box2d

Box2d is a 2D physics engine. You can install it via  and then get started as follow:

In [ ]:
!pip install gym[box2d]

In [ ]:
# Box2d Environment
env = gym.make('LunarLander-v2')
env.reset()
plt.imshow(env.render('rgb_array'))
#env.render()

## Classic control
These are a variety of classic control tasks, which would appear in a typical reinforcement learning textbook. If you didn't do the full install, you will need to run the following code to enable rendering. 

In [ ]:
!pip install gym[classic_control]

In [ ]:
env = gym.make('CartPole-v0')
env.reset()
plt.imshow(env.render('rgb_array'))
#env.render()

PyBullet Robotics Environments
3D physics environments like the Mujoco environments but uses the Bullet physics engine and does not require a commercial license. Works on Mac/Linux/Windows.

Learn more from Pybullet quick guide: https://docs.google.com/document/d/10sXEhzFRSnvFcl3XxNGhnD4N2SedqwdAvK3dsihxVUA/edit#heading=h.wz5to0x8kqmr

In [ ]:
!pip install git+https://github.com/benelot/pybullet-gym

In [ ]:
import pybulletgym  # register PyBullet enviroments with open ai gym

#env = gym.make('HumanoidPyBulletEnv-v0')
import pybullet_envs.bullet.minitaur_gym_env as e
env = e.MinitaurBulletEnv()
env.reset()
plt.imshow(env.render('rgb_array'))


# I did not test the following two environments because of the MuJoCo License. 

## MuJoCo

MuJoCo is a physics engine which can do very detailed efficient simulations with contacts. It's not open-source, so you'll have to follow the instructions in mujoco-py to set it up. Refer the following site
https://gist.github.com/BuildingAtom/3119ac9c595324c8001a7454f23bf8c8

In [ ]:
#To run mujoco in google colab, run the following code
import os
if not os.path.exists('.mujoco_setup_complete'):
  # Get the prereqs
  !apt-get -qq update
  !apt-get -qq install -y libosmesa6-dev libgl1-mesa-glx libglfw3 libgl1-mesa-dev libglew-dev patchelf
  # Get Mujoco
  !mkdir ~/.mujoco
  !wget -q https://mujoco.org/download/mujoco210-linux-x86_64.tar.gz -O mujoco.tar.gz
  !tar -zxf mujoco.tar.gz -C "$HOME/.mujoco"
  !rm mujoco.tar.gz
  # Add it to the actively loaded path and the bashrc path (these only do so much)
  !echo 'export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$HOME/.mujoco/mujoco210/bin' >> ~/.bashrc 
  !echo 'export LD_PRELOAD=$LD_PRELOAD:/usr/lib/x86_64-linux-gnu/libGLEW.so' >> ~/.bashrc 
  # THE ANNOYING ONE, FORCE IT INTO LDCONFIG SO WE ACTUALLY GET ACCESS TO IT THIS SESSION
  !echo "/root/.mujoco/mujoco210/bin" > /etc/ld.so.conf.d/mujoco_ld_lib_path.conf
  !ldconfig
  # Install Mujoco-py
  !pip3 install -U 'mujoco-py<2.2,>=2.1'
  # run once
  !touch .mujoco_setup_complete

try:
  if _mujoco_run_once:
    pass
except NameError:
  _mujoco_run_once = False
if not _mujoco_run_once:
  # Add it to the actively loaded path and the bashrc path (these only do so much)
  try:
    os.environ['LD_LIBRARY_PATH']=os.environ['LD_LIBRARY_PATH'] + ':/root/.mujoco/mujoco210/bin'
  except KeyError:
    os.environ['LD_LIBRARY_PATH']='/root/.mujoco/mujoco210/bin'
  try:
    os.environ['LD_PRELOAD']=os.environ['LD_PRELOAD'] + ':/usr/lib/x86_64-linux-gnu/libGLEW.so'
  except KeyError:
    os.environ['LD_PRELOAD']='/usr/lib/x86_64-linux-gnu/libGLEW.so'
  # presetup so we don't see output on first env initialization
  import mujoco_py
  _mujoco_run_once = True

Refer here https://github.com/openai/mujoco-py

In [ ]:
import mujoco_py
import os
mj_path = mujoco_py.utils.discover_mujoco()
xml_path = os.path.join(mj_path, 'model', 'humanoid.xml')
model = mujoco_py.load_model_from_path(xml_path)
sim = mujoco_py.MjSim(model)

print(sim.data.qpos)

sim.step()
print(sim.data.qpos)

In [ ]:
env = gym.make('Humanoid-v2')
env.reset()
plt.imshow(env.render('rgb_array'))
#env.render()

In [ ]:
env = gym.make('HandManipulateBlock-v0')
env.reset()
plt.imshow(env.render('rgb_array'))
#env.render()